In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define the Google Drive path
drive_path = "/content/drive/My Drive/Colab Notebooks/BoolQ_Project"
print("Files in Google Drive:")
!ls "{drive_path}"

Files in Google Drive:
boolq_finetuned_bert_best	   boolq_test_predictions_epoch3.csv  train_df_clean.csv
boolq_finetuned_bert_best_current  dev_df_clean.csv
boolq_test_predictions.csv	   test_df_clean.csv


In [ ]:
# Import libraries
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

In [ ]:
# Load cleaned DataFrames from Google Drive
drive_path = "/content/drive/My Drive/Colab Notebooks/BoolQ_Project"
train_df_clean = pd.read_csv(f"{drive_path}/train_df_clean.csv")
dev_df_clean = pd.read_csv(f"{drive_path}/dev_df_clean.csv")
test_df_clean = pd.read_csv(f"{drive_path}/test_df_clean.csv")
print("Loaded cleaned DataFrames:")
print("Train shape:", train_df_clean.shape)
print("Dev shape:", dev_df_clean.shape)
print("Test shape:", test_df_clean.shape)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Define the BoolQDataset class
class BoolQDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, label_column='answer', is_test=False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_column = label_column
        self.is_test = is_test  # Flag to indicate if this is the test set

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = str(self.data.iloc[idx]['question'])
        passage = str(self.data.iloc[idx]['passage'])

        # Handle labels (only for train/dev, not for test)
        if not self.is_test:
            label = 1 if self.data.iloc[idx][self.label_column] else 0
            label_tensor = torch.tensor(label, dtype=torch.long)
        else:
            label_tensor = torch.tensor(0, dtype=torch.long)  # Dummy label for test set (not used)

        encoding = self.tokenizer(
            question,
            passage,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation='longest_first',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label_tensor
        }

# Load the tokenizer (already done, but included for completeness)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create datasets
label_column = 'answer'  # Confirmed as the correct column name
train_dataset = BoolQDataset(train_df_clean, tokenizer, label_column=label_column)
dev_dataset = BoolQDataset(dev_df_clean, tokenizer, label_column=label_column)
test_dataset = BoolQDataset(test_df_clean, tokenizer, label_column=label_column, is_test=True)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
dev_loader = DataLoader(dev_dataset, sampler=SequentialSampler(dev_dataset), batch_size=batch_size)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

print(f"Train batches: {len(train_loader)}")
print(f"Dev batches: {len(dev_loader)}")
print(f"Test batches: {len(test_loader)}")

Loaded cleaned DataFrames:
Train shape: (9427, 4)
Dev shape: (3270, 4)
Test shape: (3245, 3)
Train batches: 1179
Dev batches: 409
Test batches: 406


In [ ]:
# Copy the model from Google Drive to Colab
drive_path = "/content/drive/My Drive/Colab Notebooks/BoolQ_Project"
!cp -r "{drive_path}/boolq_finetuned_bert_best" /content/

# Verify the model files
print("Model files in /content/boolq_finetuned_bert_best:")
!ls /content/boolq_finetuned_bert_best

# Load the model
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('/content/boolq_finetuned_bert_best')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("BERT model loaded successfully.")

Model files in /content/boolq_finetuned_bert_best:
config.json  model.safetensors	special_tokens_map.json  tokenizer_config.json	vocab.txt
BERT model loaded successfully.


In [ ]:
# Print the column names of each DataFrame
print("Train DataFrame columns:", train_df_clean.columns)
print("Dev DataFrame columns:", dev_df_clean.columns)
print("Test DataFrame columns:", test_df_clean.columns)

# Print a sample of each DataFrame
print("\nTrain DataFrame sample:\n", train_df_clean.head())
print("\nDev DataFrame sample:\n", dev_df_clean.head())
print("\nTest DataFrame sample:\n", test_df_clean.head())

Train DataFrame columns: Index(['question', 'title', 'answer', 'passage'], dtype='object')
Dev DataFrame columns: Index(['question', 'title', 'answer', 'passage'], dtype='object')
Test DataFrame columns: Index(['question', 'title', 'passage'], dtype='object')

Train DataFrame sample:
                                             question  \
0    do iran and afghanistan speak the same language   
1  do good samaritan laws protect those who help ...   
2  is windows movie maker part of windows essentials   
3  is confectionary sugar the same as powdered sugar   
4         is elder scrolls online the same as skyrim   

                      title  answer  \
0          Persian language    True   
1        Good Samaritan law    True   
2       Windows Movie Maker    True   
3            Powdered sugar    True   
4  The Elder Scrolls Online   False   

                                             passage  
0  Persian (/ˈpɜːrʒən, -ʃən/), also known by its ...  
1  Good Samaritan laws offer leg

In [ ]:
class BoolQDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, label_column='answer', is_test=False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_column = label_column
        self.is_test = is_test  # Flag to indicate if this is the test set

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = str(self.data.iloc[idx]['question'])
        passage = str(self.data.iloc[idx]['passage'])

        # Handle labels (only for train/dev, not for test)
        if not self.is_test:
            label = 1 if self.data.iloc[idx][self.label_column] else 0
            label_tensor = torch.tensor(label, dtype=torch.long)
        else:
            label_tensor = torch.tensor(0, dtype=torch.long)  # Dummy label for test set (not used)

        encoding = self.tokenizer(
            question,
            passage,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation='longest_first',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label_tensor
        }

# Create datasets with the correct label column name
# Replace 'answer' with the actual column name after inspecting the DataFrames
label_column = 'answer'  # Update this based on the output of Step 1
train_dataset = BoolQDataset(train_df_clean, tokenizer, label_column=label_column)
dev_dataset = BoolQDataset(dev_df_clean, tokenizer, label_column=label_column)
test_dataset = BoolQDataset(test_df_clean, tokenizer, label_column=label_column, is_test=True)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
dev_loader = DataLoader(dev_dataset, sampler=SequentialSampler(dev_dataset), batch_size=batch_size)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

print(f"Train batches: {len(train_loader)}")
print(f"Dev batches: {len(dev_loader)}")
print(f"Test batches: {len(test_loader)}")

Train batches: 1179
Dev batches: 409
Test batches: 406


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# Evaluate on the dev set
model.eval()
dev_preds, dev_labels = [], []
total_dev_loss = 0
with torch.no_grad():
    for batch in tqdm(dev_loader, desc="Evaluating BERT"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        total_dev_loss += outputs.loss.item()
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        dev_preds.extend(preds.cpu().numpy())
        dev_labels.extend(labels.cpu().numpy())

avg_dev_loss = total_dev_loss / len(dev_loader)
accuracy = accuracy_score(dev_labels, dev_preds)
f1 = f1_score(dev_labels, dev_preds)

print(f"BERT Validation Loss: {avg_dev_loss:.4f}")
print(f"BERT Dev Accuracy: {accuracy:.4f}")
print(f"BERT Dev F1: {f1:.4f}")

Evaluating BERT: 100%|██████████| 409/409 [01:33<00:00,  4.39it/s]


BERT Validation Loss: 0.7416
BERT Dev Accuracy: 0.7153
BERT Dev F1: 0.7893


In [ ]:
# Predict on the test set with BERT
model.eval()
test_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting with BERT"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())

# Add predictions to the test DataFrame
test_df_clean['predicted_answer_bert'] = [bool(pred) for pred in test_preds]
print("Sample BERT test predictions:\n", test_df_clean[['question', 'predicted_answer_bert']].head())
test_df_clean.to_csv('/content/boolq_test_predictions_bert.csv', index=False)

# Download the predictions
from google.colab import files
files.download('/content/boolq_test_predictions_bert.csv')

# Save to Google Drive
!cp /content/boolq_test_predictions_bert.csv "{drive_path}/"
print("BERT test predictions saved and downloaded.")

Predicting with BERT: 100%|██████████| 406/406 [01:37<00:00,  4.17it/s]


Sample BERT test predictions:
                                             question  predicted_answer_bert
0  is the first series 20 euro note still legal t...                   True
1  do the champions league winners get automatic ...                   True
2                  can a bull snake kill a small dog                   True
3                are all nba playoff games best of 7                  False
4  can i use my train ticket on the tram in manch...                   True


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BERT test predictions saved and downloaded.


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# Load RoBERTa tokenizer and model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
roberta_model.to(device)
print("RoBERTa model loaded successfully.")

# Create datasets with RoBERTa tokenizer
train_dataset_roberta = BoolQDataset(train_df_clean, roberta_tokenizer, label_column='answer')
dev_dataset_roberta = BoolQDataset(dev_df_clean, roberta_tokenizer, label_column='answer')
train_loader_roberta = DataLoader(train_dataset_roberta, sampler=RandomSampler(train_dataset_roberta), batch_size=8)
dev_loader_roberta = DataLoader(dev_dataset_roberta, sampler=SequentialSampler(dev_dataset_roberta), batch_size=8)

# Optimizer and scheduler
optimizer = AdamW(roberta_model.parameters(), lr=1e-5, weight_decay=0.1)
num_training_steps = len(train_loader_roberta) * 3
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTa model loaded successfully.


In [ ]:
# Training loop for RoBERTa
best_f1 = 0
best_epoch = 0
for epoch in range(3):
    print(f"\nRoBERTa Epoch {epoch + 1}/3")
    roberta_model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_loader_roberta, desc="Training RoBERTa")):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = roberta_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(roberta_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if (step + 1) % 50 == 0:
            print(f"Step {step + 1}/{len(train_loader_roberta)}, Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader_roberta)
    print(f"RoBERTa Average Training Loss: {avg_train_loss:.4f}")

    # Evaluate RoBERTa
    roberta_model.eval()
    dev_preds, dev_labels = [], []
    total_dev_loss = 0
    with torch.no_grad():
        for batch in tqdm(dev_loader_roberta, desc="Evaluating RoBERTa"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = roberta_model(input_ids, attention_mask=attention_mask, labels=labels)
            total_dev_loss += outputs.loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            dev_preds.extend(preds.cpu().numpy())
            dev_labels.extend(labels.cpu().numpy())

    avg_dev_loss = total_dev_loss / len(dev_loader_roberta)
    accuracy = accuracy_score(dev_labels, dev_preds)
    f1 = f1_score(dev_labels, dev_preds)
    print(f"RoBERTa Validation Loss: {avg_dev_loss:.4f}")
    print(f"RoBERTa Dev Accuracy: {accuracy:.4f}")
    print(f"RoBERTa Dev F1: {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_epoch = epoch + 1
        roberta_model.save_pretrained("/content/boolq_finetuned_roberta_best")
        roberta_tokenizer.save_pretrained("/content/boolq_finetuned_roberta_best")
        print(f"New best RoBERTa model saved with F1 {best_f1:.4f}")
        drive_path = "/content/drive/My Drive/Colab Notebooks/BoolQ_Project"
        !cp -r /content/boolq_finetuned_roberta_best "{drive_path}/"
    elif epoch - best_epoch >= 1:
        print(f"Early stopping at epoch {epoch + 1}. Best F1: {best_f1:.4f}")
        break

print(f"RoBERTa Best F1: {best_f1:.4f} at epoch {best_epoch}")


RoBERTa Epoch 1/3


Training RoBERTa:   4%|▍         | 50/1179 [00:37<16:41,  1.13it/s]

Step 50/1179, Loss: 0.5936


Training RoBERTa:   8%|▊         | 100/1179 [01:13<15:39,  1.15it/s]

Step 100/1179, Loss: 0.7439


...

Step 1150/1179, Loss: 0.2994


Training RoBERTa: 100%|██████████| 1179/1179 [14:17<00:00,  1.38it/s]


RoBERTa Average Training Loss: 0.3822


Evaluating RoBERTa: 100%|██████████| 409/409 [01:26<00:00,  4.73it/s]


RoBERTa Validation Loss: 0.5660
RoBERTa Dev Accuracy: 0.7835
RoBERTa Dev F1: 0.8261
New best RoBERTa model saved with F1 0.8261
RoBERTa Best F1: 0.8261 at epoch 3


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# Load DistilBERT tokenizer and model
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
distilbert_model.to(device)
print("DistilBERT model loaded successfully.")

# Create datasets with DistilBERT tokenizer
train_dataset_distilbert = BoolQDataset(train_df_clean, distilbert_tokenizer, label_column='answer')
dev_dataset_distilbert = BoolQDataset(dev_df_clean, distilbert_tokenizer, label_column='answer')
train_loader_distilbert = DataLoader(train_dataset_distilbert, sampler=RandomSampler(train_dataset_distilbert), batch_size=8)
dev_loader_distilbert = DataLoader(dev_dataset_distilbert, sampler=SequentialSampler(dev_dataset_distilbert), batch_size=8)

# Optimizer and scheduler
optimizer = AdamW(distilbert_model.parameters(), lr=1e-5, weight_decay=0.1)
num_training_steps = len(train_loader_distilbert) * 3
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBERT model loaded successfully.


In [ ]:
# Training loop for DistilBERT
best_f1 = 0
best_epoch = 0
for epoch in range(3):
    print(f"\nDistilBERT Epoch {epoch + 1}/3")
    distilbert_model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_loader_distilbert, desc="Training DistilBERT")):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = distilbert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(distilbert_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if (step + 1) % 50 == 0:
            print(f"Step {step + 1}/{len(train_loader_distilbert)}, Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader_distilbert)
    print(f"DistilBERT Average Training Loss: {avg_train_loss:.4f}")

    # Evaluate DistilBERT
    distilbert_model.eval()
    dev_preds, dev_labels = [], []
    total_dev_loss = 0
    with torch.no_grad():
        for batch in tqdm(dev_loader_distilbert, desc="Evaluating DistilBERT"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = distilbert_model(input_ids, attention_mask=attention_mask, labels=labels)
            total_dev_loss += outputs.loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            dev_preds.extend(preds.cpu().numpy())
            dev_labels.extend(labels.cpu().numpy())

    avg_dev_loss = total_dev_loss / len(dev_loader_distilbert)
    accuracy = accuracy_score(dev_labels, dev_preds)
    f1 = f1_score(dev_labels, dev_preds)
    print(f"DistilBERT Validation Loss: {avg_dev_loss:.4f}")
    print(f"DistilBERT Dev Accuracy: {accuracy:.4f}")
    print(f"DistilBERT Dev F1: {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_epoch = epoch + 1
        distilbert_model.save_pretrained("/content/boolq_finetuned_distilbert_best")
        distilbert_tokenizer.save_pretrained("/content/boolq_finetuned_distilbert_best")
        print(f"New best DistilBERT model saved with F1 {best_f1:.4f}")
        drive_path = "/content/drive/My Drive/Colab Notebooks/BoolQ_Project"
        !cp -r /content/boolq_finetuned_distilbert_best "{drive_path}/"
    elif epoch - best_epoch >= 1:
        print(f"Early stopping at epoch {epoch + 1}. Best F1: {best_f1:.4f}")
        break

print(f"DistilBERT Best F1: {best_f1:.4f} at epoch {best_epoch}")


DistilBERT Epoch 1/3


Training DistilBERT:   4%|▍         | 51/1179 [00:17<06:16,  3.00it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Step 50/1179, Loss: 0.6614


Training DistilBERT:   9%|▊         | 101/1179 [00:35<06:06,  2.94it/s]

Step 100/1179, Loss: 0.5987


...

Step 1150/1179, Loss: 0.6572


Training DistilBERT: 100%|██████████| 1179/1179 [07:09<00:00,  2.75it/s]


DistilBERT Average Training Loss: 0.4672


Evaluating DistilBERT: 100%|██████████| 409/409 [00:50<00:00,  8.13it/s]


DistilBERT Validation Loss: 0.6071
DistilBERT Dev Accuracy: 0.7073
DistilBERT Dev F1: 0.7735
New best DistilBERT model saved with F1 0.7735
DistilBERT Best F1: 0.7735 at epoch 3


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import torch

# Load the best RoBERTa model
roberta_model = RobertaForSequenceClassification.from_pretrained('/content/boolq_finetuned_roberta_best')
roberta_tokenizer = RobertaTokenizer.from_pretrained('/content/boolq_finetuned_roberta_best')
roberta_model.to(device)
print("Best RoBERTa model loaded successfully.")

# Create test dataset with RoBERTa tokenizer
test_dataset_roberta = BoolQDataset(test_df_clean, roberta_tokenizer, label_column='answer', is_test=True)
test_loader_roberta = DataLoader(test_dataset_roberta, sampler=SequentialSampler(test_dataset_roberta), batch_size=8)

# Predict on the test set with RoBERTa
roberta_model.eval()
test_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader_roberta, desc="Predicting with RoBERTa"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = roberta_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())

# Add predictions to the test DataFrame
test_df_clean['predicted_answer_roberta'] = [bool(pred) for pred in test_preds]
print("Sample RoBERTa test predictions:\n", test_df_clean[['question', 'predicted_answer_roberta']].head())
test_df_clean.to_csv('/content/boolq_test_predictions_roberta.csv', index=False)

# Download the predictions
from google.colab import files
files.download('/content/boolq_test_predictions_roberta.csv')

# Save to Google Drive
drive_path = "/content/drive/My Drive/Colab Notebooks/BoolQ_Project"
!cp /content/boolq_test_predictions_roberta.csv "{drive_path}/"
print("RoBERTa test predictions saved and downloaded.")

Best RoBERTa model loaded successfully.


Predicting with RoBERTa: 100%|██████████| 406/406 [01:27<00:00,  4.66it/s]


Sample RoBERTa test predictions:
                                             question  predicted_answer_roberta
0  is the first series 20 euro note still legal t...                     False
1  do the champions league winners get automatic ...                      True
2                  can a bull snake kill a small dog                      True
3                are all nba playoff games best of 7                      True
4  can i use my train ticket on the tram in manch...                      True


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

RoBERTa test predictions saved and downloaded.
